# Imports

In [ ]:
import pandas as pd 
import numpy as np

### Base com atributos numéricos

In [ ]:
# Base tirada de https://archive.ics.uci.edu/ml/datasets/Balance+Scale

balance = pd.read_csv('databases/balance-scale.csv',
                      header=None, 
                      names=["Pendência", "Peso_E", "Distância_E", "Peso_D", "Distância_D"])

# Substituindo a string de pendência para valores compreenciveis
balance = balance.replace({
    "B": "Balanceado", 
    "L": "Esquerda", 
    "R": "Direita", 
})

balance

In [ ]:
balance.describe()

In [ ]:
balance.dtypes

## Pré-Processamento Inicial

In [ ]:
# Sem necessidade pois os atributos da base são numéricos

# balance = pd.get_dummies(balance, 
#                          columns=[], 
#                          prefix=[])
# balance

In [ ]:
# Sem necessidade pois os atributos da base são apenas numericos

# from sklearn.preprocessing import OrdinalEncoder
# atributos = [
#                          "Peso_E", "Distância_E", "Peso_D", "Distância_D"
#                       ]
# enc = OrdinalEncoder()
# enc.fit(balance[atributos])
# balance[atributos] = enc.transform(balance[atributos])
# balance

In [ ]:
from sklearn.preprocessing import StandardScaler

atributos = ["Peso_E", "Distância_E", "Peso_D", "Distância_D"]

balance[atributos] = StandardScaler().fit_transform(balance[atributos])
#balance = pd.DataFrame(balance)

balance


## Clusterizando a Balança
##### Selecionando o X e y

In [ ]:
# Selecionando o X e o y


# X = jogo_da_velha[atributos]
X = balance.iloc[:, 1:].values # Atributos do Data Frame do Pandas
y = balance.iloc[:, :1].values # Classes do Data Frame do Pandas

y = np.squeeze(y) # Converter em um unico Array quando usar o dataFrame do Pandas

y_unique = np.unique(y)

# Atualizando o eixo Y
for i in range(len(y_unique)):
    y = [i if y_unique[i] == e else e for e in y]
y = np.array(y)

y_unique # Valores unicos de classes

##### Exemplos de Algoritmos para cálculos de distâncias

In [ ]:
# from sklearn.metrics.pairwise import pairwise_distances, cosine_similarity


# eucl = pairwise_distances(X[0].reshape(1, -1), X[1].reshape(1, -1), 'euclidean')

# manh = pairwise_distances(X[0].reshape(1, -1), X[1].reshape(1, -1), 'manhattan')

# ass_cos = cosine_similarity(X[0].reshape(1, -1), X[1].reshape(1, -1))

# print("Euclidiano: ", eucl)
# print("Manhattan: ", manh)
# print("Assimilaridade do Cosseno: ", ass_cos)

##### PCA

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_reduced = pca.fit_transform(X)

print('Projecting %d-dimensional data to 2D' % X.shape[1])

def plot_g(pyplot, title, colors, n_classes):
    pyplot.scatter(X_reduced[:, 0], X_reduced[:, 1], c=colors, 
             edgecolor='none', alpha=0.7, s=40,
             cmap=plt.cm.get_cmap('nipy_spectral', n_classes)
           )
    pyplot.colorbar()
    pyplot.title(title)

plt.figure(figsize=(10, 8)) # Tamanho da Figura
plot_g(plt, 'Projeção 2D do Dados em PCA', y, len(y_unique))
plt.show()


##### KMeans com a base original e com o PCA

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=len(y_unique)).fit(X)
labels = kmeans.labels_

kmeans_PCA = KMeans(n_clusters=len(y_unique)).fit(X_reduced)
labels_PCA = kmeans_PCA.labels_


##### Plotando os Gráficos 

In [ ]:

plt.figure(figsize=(10*2, 7))

plt.subplot(1, 2, 1) # row 1, col 2 index 1
plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=labels, 
            edgecolor='none', alpha=0.7, s=40,
            cmap=plt.cm.get_cmap('nipy_spectral', len(y_unique)))
plt.colorbar()
plt.title('Projeção KMeans com X Original')

plt.subplot(1, 2, 2) # row 1, col 2 index 1
plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=labels_PCA, 
            edgecolor='none', alpha=0.7, s=40,
            cmap=plt.cm.get_cmap('nipy_spectral', len(y_unique)))
plt.colorbar()
plt.title('Projeção KMeans com X PCA')

plt.show()

##### Comparando o KMeans, KMeans PCA e o y Original

In [ ]:
print("Comparação: ")

unique, counts = np.unique(labels, return_counts=True) # KMeans
print("KMeans X: " , dict(zip(unique, counts)))

unique, counts = np.unique(labels_PCA, return_counts=True) # KMeans PCA
print("KM.. PCA: ", dict(zip(unique, counts)))

unique, counts = np.unique(y, return_counts=True) # Original
print("Original: ", dict(zip(unique, counts)))


##### Usando as metricas de avaliação para comparar os resultados com e sem o PCA

##### Sem PCA

In [ ]:
from sklearn import metrics
from sklearn.metrics.cluster import v_measure_score

ari = metrics.adjusted_rand_score(y, labels)
silh = metrics.silhouette_score(X, labels, metric='euclidean')
ami = metrics.adjusted_mutual_info_score(y, labels)
vm = v_measure_score(y, labels)

##### Com PCA

In [ ]:
ari_pca = metrics.adjusted_rand_score(y, labels_PCA)
silh_pca = metrics.silhouette_score(X_reduced, labels_PCA, metric='euclidean')
ami_pca = metrics.adjusted_mutual_info_score(y, labels_PCA)
vm_pca = v_measure_score(y, labels_PCA)

##### Tabela de comparação dos resultados

In [ ]:
data = {
# "Base de dados": [], 
"Pré-processamento":['Com PCA', 'Sem PCA'], 
"SILHUETA":[silh_pca, silh], 
"ARI": [ari_pca, ari], 
"AMI": [ami_pca, ami], 
"V-M":[vm_pca, vm]
}

print(pd.DataFrame(data))